In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 데이터셋 경로 설정
data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

# 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = ''.join(e for e in text if e.isalnum() or e.isspace())  # 특수문자 제거
    return text

# 커스텀 데이터셋 클래스 정의
class ImageTextDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.data = []

        # 폴더 내 파일 목록 가져오기
        file_names = [f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.jpg')]  # .jpg 파일 기준
        unique_ids = set(file_names)  # 중복 제거

        for file_id in unique_ids:
            # 이미지와 텍스트 파일 경로 생성
            image_file = os.path.join(folder_path, f"{file_id}.jpg")
            text_file = os.path.join(folder_path, f"{file_id}.txt")

            # 이미지와 텍스트 파일이 모두 있는 경우 로드
            if os.path.exists(image_file) and os.path.exists(text_file):
                self.data.append((image_file, text_file))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_file, text_file = self.data[idx]

        # 이미지 로드 및 전처리
        image = cv2.imread(image_file)
        if image is None:
            raise ValueError(f"이미지를 로드할 수 없습니다: {image_file}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환

        if self.transform:
            image = self.transform(image)
        else:
            # 기본 변환 적용
            image = cv2.resize(image, (224, 224))
            image = image / 255.0  # 정규화
            image = torch.tensor(image, dtype=torch.float32)
            image = image.permute(2, 0, 1)  # (H, W, C) -> (C, H, W)

        # 텍스트 로드 및 전처리
        with open(text_file, 'r') as file:
            raw_text = file.read().strip()
        text = preprocess_text(raw_text)

        return image, text

# 데이터셋 및 데이터로더 생성
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageTextDataset(data_path, transform=transform)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

## 2. 모델과 라벨 1차 평가

In [11]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from sklearn.metrics import classification_report

# 1. 라벨 파일 로드 및 전처리
label_file_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt'

# 라벨 파일 로드
label_df = pd.read_csv(label_file_path, sep='\t', header=0)

# 컬럼 이름 지정 (필요에 따라)
label_df.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']

# 감성 라벨 매핑
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

# 각 샘플에 대한 라벨러들의 라벨 수집
all_labels = []
sample_ids = label_df['ID'].tolist()
for idx, row in label_df.iterrows():
    labels = []
    for annotator in ['Annotator1', 'Annotator2', 'Annotator3']:
        text_label = row[annotator].split(',')[0].strip()  # 텍스트 라벨 추출
        labels.append(text_label)
    # 라벨 매핑 적용
    mapped_labels = [label_mapping[label] for label in labels]
    all_labels.append(mapped_labels)  # 각 샘플에 대한 세 명의 라벨러 라벨 저장

In [13]:
# 2. 모델 로드
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # 예시 모델 이름
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# GPU 사용 설정 (가능한 경우)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  # 평가 모드로 설정

# 3. 예측 수행
predicted_labels = []
valid_sample_ids = []
valid_all_labels = []

# 진행 상황 모니터링을 위한 설정
total_samples = len(sample_ids)
print(f"총 샘플 수: {total_samples}")


^C


In [18]:
# 각 샘플에 대해 예측 수행
for idx, (sample_id, labels) in enumerate(zip(sample_ids, all_labels)):
    if idx % 100 == 0:
        print(f"{idx}/{total_samples} 처리 중...")

    text_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/', f"{sample_id}.txt")
    if os.path.exists(text_file):
        # 텍스트 로드 및 전처리
        with open(text_file, 'r', encoding='utf-8') as file:
            raw_text = file.read().strip()

        # 텍스트 토크나이즈
        inputs = tokenizer(
            raw_text,
            return_tensors='pt',
            truncation=True,
            padding=True,
            max_length=128
        )
        inputs = {key: val.to(device) for key, val in inputs.items()}

        # 모델 예측
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
            predicted_labels.append(predicted_class)
            valid_sample_ids.append(sample_id)
            valid_all_labels.append(labels)
    else:
        print(f"{sample_id}.txt 파일을 찾을 수 없습니다.")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [19]:
# 4. 예측과 라벨 비교 (최대 일치율 계산)
correct = 0
total = len(predicted_labels)

for pred, true_labels in zip(predicted_labels, valid_all_labels):
    if pred in true_labels:
        correct += 1

accuracy = correct / total
print(f"모델의 최대 일치율: {accuracy * 100:.2f}%")

# 추가로 일치하는 샘플과 일치하지 않는 샘플의 개수 출력
print(f"일치하는 샘플 수: {correct}")
print(f"일치하지 않는 샘플 수: {total - correct}")

# 선택적으로, 각 클래스별로 정확도를 계산할 수 있습니다.
from sklearn.metrics import classification_report

# 라벨을 하나의 리스트로 펼치기 (각 샘플마다 세 개의 라벨이 있으므로, 비교를 위해 같은 라벨을 세 번씩 복제)
flattened_true_labels = []
for labels in valid_all_labels:
    # 모델의 예측과 비교하기 위해 각 샘플마다 예측 라벨을 세 번 복제
    flattened_true_labels.extend(labels)

flattened_predicted_labels = []
for pred in predicted_labels:
    # 각 예측 라벨을 세 번 복제하여 flattened_true_labels와 길이를 맞춤
    flattened_predicted_labels.extend([pred] * 3)

# 분류 보고서 출력
print(classification_report(flattened_true_labels, flattened_predicted_labels, target_names=['Negative', 'Neutral', 'Positive']))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
17424.txt 파일을 찾을 수 없습니다.
17425.txt 파일을 찾을 수 없습니다.
17426.txt 파일을 찾을 수 없습니다.
17427.txt 파일을 찾을 수 없습니다.
17428.txt 파일을 찾을 수 없습니다.
17429.txt 파일을 찾을 수 없습니다.
17430.txt 파일을 찾을 수 없습니다.
17431.txt 파일을 찾을 수 없습니다.
17432.txt 파일을 찾을 수 없습니다.
17433.txt 파일을 찾을 수 없습니다.
17434.txt 파일을 찾을 수 없습니다.
17435.txt 파일을 찾을 수 없습니다.
17436.txt 파일을 찾을 수 없습니다.
17437.txt 파일을 찾을 수 없습니다.
17438.txt 파일을 찾을 수 없습니다.
17439.txt 파일을 찾을 수 없습니다.
17440.txt 파일을 찾을 수 없습니다.
17441.txt 파일을 찾을 수 없습니다.
17442.txt 파일을 찾을 수 없습니다.
17443.txt 파일을 찾을 수 없습니다.
17444.txt 파일을 찾을 수 없습니다.
17445.txt 파일을 찾을 수 없습니다.
17446.txt 파일을 찾을 수 없습니다.
17447.txt 파일을 찾을 수 없습니다.
17448.txt 파일을 찾을 수 없습니다.
17449.txt 파일을 찾을 수 없습니다.
17451.txt 파일을 찾을 수 없습니다.
17452.txt 파일을 찾을 수 없습니다.
17453.txt 파일을 찾을 수 없습니다.
17454.txt 파일을 찾을 수 없습니다.
17455.txt 파일을 찾을 수 없습니다.
17456.txt 파일을 찾을 수 없습니다.
17457.txt 파일을 찾을 수 없습니다.
17458.txt 파일을 찾을 수 없습니다.
17459.txt 파일을 찾을 수 없습니다.
17460.txt 파일을 찾을 수 없습니다.
17461.txt 파일을 찾을 수 없습니다.
17462.txt 파일을 찾을 수 없습니다.
17463.txt 파일을 